In [25]:
from underthesea import sent_tokenize
import os
from tqdm import tqdm
import re

def split_text_into_chunks(text, chunk_size=100, window_size=50):
    """Split a long text into multiple chunks (passages) with managable sizes.
    
    Args:
        chunk_size (int): Maximum size of a chunk.
        window_size (int): Decide how many words are overlapped between two consecutive chunks. Basically #overlapped_words = chunk_size - window_size.
    Returns:
        str: Multiple chunks of text splitted from initial document text.
    """
    words = text.split()
    num_words = len(words)
    chunks = []
    start_idx = 0

    while True:
        end_idx = start_idx + chunk_size
        chunk = " ".join(words[start_idx:end_idx])
        chunks.append(chunk)
        if end_idx >= num_words:
            break
        start_idx += window_size

    return chunks

def split_doc(doc, size = 325):
    """
    args: A document and a chunk size
    return: list of sub-doc that have size <= chunk size, make sure not split in the middle sentences.    
    """

    placeholder = "<NEWLINE>"

    context = doc.strip()
    text_with_placeholders = context.replace("\n", f" {placeholder} ")

    words = re.split(r'[\t ]+', text_with_placeholders)
    result = [word if word != placeholder else "\n" for word in words]

    # print(result)
    lenght = len(result)
    index = 0
    while index < lenght:
        if (result[index] == "\n") and (result[index + 1] == "\n"):
            lenght -= 1
            result.pop(index+1)
        else:
            index += 1
    # print(result)
    count = 0
    # for each in result:
    #     if each == "\n":
    #         count += 1

    # print(count)
    # # print(result[:100])
    chunks = []

    save_idx = 0
    while len(result) > size :
        for i in range(size,-1,-1):
            if result[i] == "\n":
                save_idx = i
                break
        # print(result[:save_idx])
        chunks.append(" ".join(result[:save_idx]))
        result = result[save_idx+1:]
    chunks.append(" ".join(result))
    return chunks

def get_corpus(data_dir):
    """Transform a corpus of documents into a corpus of passages.
    
    Args:
        data_dir (str): directory that contains .txt files, each file contains text content of a wikipedia page.
    Returns:
        str: A corpus of chunks splitted from multiple initial documents. Each chunk will contain information about (id, title, passage)
    """
    corpus = []
    meta_corpus = []
    # data_dir = "./all_content/"
    filenames = os.listdir(data_dir)
    filenames = sorted(filenames, key=lambda x: int(re.search(r'Điều (\d+)', x).group(1)))
    
    _id = 1002
    # docs = {}
    for filename in tqdm(filenames):
        filepath = data_dir + filename
        title = filename.strip(".txt")
        with open(filepath, "r") as f:
            text = f.read().split("\n")
            title = text[0]
            text = "\n".join(text[1:]).strip()
            # docs[title] = text
            # text = text.lstrip(title).strip()   #modify thisss........

            # No overlap.
            chunks = split_doc(text, size = 230)
            chunks = [f"{chunk}" for chunk in chunks]
            meta_chunks = [{
                "id": _id + i,
                "len": len(chunks[i].split()),
                "title": title,
                "passage": title+ "\n" +chunks[i]   #modify thissss.....
            } for i in range(len(chunks))]
            _id += len(chunks)
            corpus.extend(chunks)
            meta_corpus.extend(meta_chunks)
    return meta_corpus

In [26]:
meta_corpus = get_corpus("/home/alex/FPT/DPL302m/project/Data_mining/Data/Enterprise_Law/Final_ENT_Law/split_guide_docs/temp/6/")
print(f">>> Corpus size: {len(meta_corpus)}")
print(f">>> Example passage")
meta_corpus[5]

100%|██████████| 45/45 [00:00<00:00, 1392.37it/s]

>>> Corpus size: 113
>>> Example passage


{'id': 1007,
 'len': 163,
 'title': 'Điều 4. Giải thích từ ngữ',
 'passage': 'Điều 4. Giải thích từ ngữ\nNgoài các từ ngữ quy định tại Luật Chứng khoán số 54/2019/QH14, trong Nghị định này, các từ ngữ dưới đây được hiểu như sau: \n 1. “Trái phiếu doanh nghiệp” là loại chứng khoán có kỳ hạn từ 01 năm trở lên do doanh nghiệp phát hành, xác nhận quyền và lợi ích hợp pháp của người sở hữu đối với một phần nợ của doanh nghiệp phát hành. \n 2. “Trái phiếu doanh nghiệp xanh” là trái phiếu doanh nghiệp được phát hành để đầu tư cho dự án thuộc lĩnh vực bảo vệ môi trường, dự án mang lại lợi ích về môi trường theo quy định của pháp luật bảo vệ môi trường. \n 3. “Trái phiếu chuyển đổi” là loại hình trái phiếu do công ty cổ phần phát hành, có thể chuyển đổi thành cổ phiếu phổ thông của chính doanh nghiệp phát hành theo điều kiện, điều khoản đã được xác định tại phương án phát hành trái phiếu.'}

In [27]:
import json
with open("/home/alex/FPT/DPL302m/project/Data_mining/Data/Enterprise_Law/Final_ENT_Law/chunk_split/final_chunk/6.jsonl", 'w', encoding="utf-8") as file:
    json.dump(meta_corpus, file, indent=4, ensure_ascii=False)